## generate abilene training dataset for TELGEN

In [1]:
from solver.linprog import linprog
from tqdm import tqdm

import gzip
import pickle
import torch
from scipy.linalg import LinAlgWarning
from scipy.optimize._optimize import OptimizeWarning
# from scipy.optimize import OptimizeWarning
import warnings
import numpy as np
from functools import partial
import random
import pickle 
import json

In [2]:
root = 'raw/'

### Resource Allocation

#### input: for one graph   
#### G(V, E, c): random graph (strongly connected)
#### (s, t, d) \in [S, T, D]  
#### for every (s, t, d), there is a set p \in Pd (k-shortest path algorithm (4/5/6))

In [3]:
import networkx as nx
import matplotlib.pyplot as plt

### generate and save connected and directed ASN different nodes and p

In [4]:
f = open(root+'abilene_graph/ABILENE.json')
G = json.load(f)

abilene_graph = nx.DiGraph()
abilene_graph.add_nodes_from([i['id'] for i in G['nodes']])
abilene_graph.add_edges_from([(i['source'], i['target']) for i in G['links']])

print('Strongly connected:', nx.is_strongly_connected(abilene_graph))
print('# of nodes and edges:', abilene_graph.number_of_nodes(), abilene_graph.number_of_edges())
print('Weighted:', nx.is_weighted(abilene_graph))

abilene_graph_weight = nx.DiGraph()
abilene_graph_weight.add_nodes_from([i['id'] for i in G['nodes']])
for i in G['links']:
    abilene_graph_weight.add_edge(i['source'], i['target'], weight=float(i['capacity']))
print('Strongly connected:', nx.is_strongly_connected(abilene_graph_weight))
print('# of nodes and edges:', abilene_graph_weight.number_of_nodes(), abilene_graph_weight.number_of_edges())
print('Weighted:', nx.is_weighted(abilene_graph_weight))

# nx.draw(asn_graph, with_labels=True, node_color='lightgreen', arrows=True)

Strongly connected: True
# of nodes and edges: 12 30
Weighted: False
Strongly connected: True
# of nodes and edges: 12 30
Weighted: True


In [5]:
# check b4 link capacities
cap = []
for i in G['links']:
    cap.append(float(i['capacity']))
max(cap), min(cap), type(cap[0])

(9.92, 2.48, float)

### generate k-shortest path

In [6]:
from itertools import islice
def k_shortest_paths(G, source, target, k, weight=None):
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))

### Read abilene and their capacities and load as a group

In [7]:
root = 'raw/'
f = open(root+'abilene_graph/ABILENE.json')
G = json.load(f)

abilene_graph = nx.DiGraph()
abilene_graph.add_nodes_from([i['id'] for i in G['nodes']])
abilene_graph.add_edges_from([(i['source'], i['target']) for i in G['links']])

abilene_graph_weight = nx.DiGraph()
abilene_graph_weight.add_nodes_from([i['id'] for i in G['nodes']])
for i in G['links']:
    abilene_graph_weight.add_edge(i['source'], i['target'], weight=float(i['capacity']))

group = []
group_noC = []
group.append(abilene_graph_weight)
group_noC.append(abilene_graph)

### training st pairs
### --train_start_indices 0 --train_end_indices 12096  (1-12095)
### --val_start_indices 12096 --val_end_indices 14112 (12096-14111)
### --test_start_idx 14112 --test_end_idx 16128 (14112-16128)

In [8]:
# root = '/mnt/data/fzhou/HARP/'
# path = root+'traffic_matrices/abilene/t'
# ######################## training ########################
# n = subgraph.number_of_nodes()
# STD = []
# for i in range(1, 12096):
#     with open(path+str(i)+'.pkl', 'rb') as f:
#         data = pickle.load(f)
#     std = []
#     for i in range(n):
#         for j in range(n):
#             if i == j:
#                 continue
#             elif i > j:
#                 std.append((i, j, data[(n-1)*i+j]))
#             else:       # i<j
#                 std.append((i, j, data[(n-1)*i+j-1]))
#     std = np.array(std)
#     STD.append(std)
# STD = np.array(STD)
# print('train size', STD.shape)
# np.save('abilene_train', STD)
# ######################## validation ########################
# STD = []
# for i in range(12096, 14112):
#     with open(path+str(i)+'.pkl', 'rb') as f:
#         data = pickle.load(f)
#     std = []
#     for i in range(n):
#         for j in range(n):
#             if i == j:
#                 continue
#             elif i > j:
#                 std.append((i, j, data[(n-1)*i+j]))
#             else:       # i<j
#                 std.append((i, j, data[(n-1)*i+j-1]))
#     std = np.array(std)
#     STD.append(std)
# STD = np.array(STD)
# print('valid size', STD.shape)
# np.save('abilene_valid', STD)
# ######################## testing ########################
# STD = []
# for i in range(14112, 16128):
#     with open(path+str(i)+'.pkl', 'rb') as f:
#         data = pickle.load(f)
#     std = []
#     for i in range(n):
#         for j in range(n):
#             if i == j:
#                 continue
#             elif i > j:
#                 std.append((i, j, data[(n-1)*i+j]))
#             else:       # i<j
#                 std.append((i, j, data[(n-1)*i+j-1]))
#     std = np.array(std)
#     STD.append(std)
# STD = np.array(STD)
# print('test size', STD.shape)
# np.save('abilene_test', STD)

In [9]:
# size: (# of instance, # of pairs, std values)
train_std = np.load('abilene_train.npy')
print('train size', train_std.shape)
valid_std = np.load('abilene_valid.npy')
print('valid size', valid_std.shape)
test_std = np.load('abilene_test.npy')
print('test size', test_std.shape)

train size (12095, 132, 3)
valid size (2016, 132, 3)
test size (2016, 132, 3)


In [10]:
# # get path dic for all pairs at one time

# k = 8  
# Pd = {}
# for m in range(STD.shape[1]): 
#     st = STD[0][m][0:2]
#     k_paths = k_shortest_paths(group_noC[0], int(st[0]), int(st[1]), k=k)
#     if len(k_paths) != k:
#         for i in range(8-len(k_paths)):
#             k_paths.append([-1, -1])
#     Pd[(st[0], st[1])] = k_paths
# Pd

In [11]:
# get path dic for all pairs at one time

k = 8  
Pd = {}
for m in range(train_std.shape[1]): 
    st = train_std[0][m][0:2]
    k_paths = k_shortest_paths(group_noC[0], int(st[0]), int(st[1]), k=k)
    if len(k_paths) != k:
        for i in range(8-len(k_paths)):
            k_paths.append(k_paths[0])
    Pd[(st[0], st[1])] = k_paths
Pd

{(0.0, 1.0): [[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1]],
 (0.0, 2.0): [[0, 1, 5, 2],
  [0, 1, 11, 8, 2],
  [0, 1, 4, 6, 5, 2],
  [0, 1, 4, 7, 9, 3, 6, 5, 2],
  [0, 1, 4, 7, 9, 10, 3, 6, 5, 2],
  [0, 1, 5, 2],
  [0, 1, 5, 2],
  [0, 1, 5, 2]],
 (0.0, 3.0): [[0, 1, 4, 6, 3],
  [0, 1, 5, 6, 3],
  [0, 1, 4, 7, 9, 3],
  [0, 1, 4, 7, 9, 10, 3],
  [0, 1, 11, 8, 2, 5, 6, 3],
  [0, 1, 5, 6, 4, 7, 9, 3],
  [0, 1, 5, 6, 4, 7, 9, 10, 3],
  [0, 1, 11, 8, 2, 5, 6, 4, 7, 9, 3]],
 (0.0, 4.0): [[0, 1, 4],
  [0, 1, 5, 6, 4],
  [0, 1, 11, 8, 2, 5, 6, 4],
  [0, 1, 5, 6, 3, 9, 7, 4],
  [0, 1, 5, 6, 3, 10, 9, 7, 4],
  [0, 1, 11, 8, 2, 5, 6, 3, 9, 7, 4],
  [0, 1, 11, 8, 2, 5, 6, 3, 10, 9, 7, 4],
  [0, 1, 4]],
 (0.0, 5.0): [[0, 1, 5],
  [0, 1, 4, 6, 5],
  [0, 1, 11, 8, 2, 5],
  [0, 1, 4, 7, 9, 3, 6, 5],
  [0, 1, 4, 7, 9, 10, 3, 6, 5],
  [0, 1, 5],
  [0, 1, 5],
  [0, 1, 5]],
 (0.0, 6.0): [[0, 1, 4, 6],
  [0, 1, 5, 6],
  [0, 1, 4, 7, 9, 3, 6],
  [0, 1, 11, 8, 2, 5, 6],
  [0, 1, 4, 7, 9, 10,

In [44]:
# G: G(V, E, C)                           nx.weighted.graph
# STD: demands align with ST pairs        list[([s1, t1], dmd1), ([s2, t2], dmd2),...], (string, int)
# Pd: set of paths for every st pair      dict{[s1, t1]: [([path1], cost1), ([path2], cost2)...], [s2, t2]...}
# # of std pairs = # of keys in Pd
# k: k shortest path for every (s, t, d) tuple
# MLU problem formulation:
# obj: MLU = max { \max_{p\in e} [\sum_{d\in D} d*x_d(p)] / c_e }, \forall e\in E
# s.t.: \sum_{p\in Pd} x_d(p) <= 1, \forall d \in D
#       \sum_{p\in e} \sum_{d\in D} d*x_d(p) <= c_e*MLU, \forall e\in E

# number of vars: len(STD)*k+1 

def generate_reallocation(G, STD, Pd, k):
    
    # constraint 1
    A1 = []
    for i in range(len(STD)):
        a = np.zeros(len(STD)*k+1)
        a[k*i: k*i+k] = 1
        A1.append(a)
    A1 = - np.array(A1)
    b1 = - np.ones(len(STD))

    # constrain 2
    edges_list = list(G.edges())                          
    A2 = np.zeros((G.number_of_edges(), len(STD)*k+1))

    for i in range(len(STD)):          # number of st pairs
        paths = Pd[tuple(STD[i][0:2])] # STD[i][0:2]: st pair; Pd[tuple(STD[i][0:2])]: possible paths
        for j in range(k):
            p = paths[j]   # path[j] is the path
            for n in range(len(p)-1):
                if (p[n], p[n+1]) in edges_list:   
                    A2[edges_list.index((p[n], p[n+1]))][k*i+j] = STD[i][-1] # STD[i][-1]: demand
                else:
                    continue  
    A2[:, -1] = -np.array(list(nx.get_edge_attributes(G,'weight').values()))
    for i in range(A2.shape[0]):      # normalize by the weight
        A2[i] = A2[i]/-A2[i, -1]
    b2 = np.zeros(G.number_of_edges())   

    zero_row_indices = np.where(A2.any(axis=1)==0)[0]
    A2 = np.delete(A2, zero_row_indices, axis=0)
    b2 = np.delete(b2, zero_row_indices, axis=0)
    
    # obj
    c = np.zeros(len(STD)*k+1)
    c[-1] = 1

    return A1, b1, A2, b2, c

## dataset generation

In [54]:
### gen train ####
root = 'raw/'

import time
warnings.filterwarnings("error")

random.seed(2024)
np.random.seed(2024)


pkg_idx = 0              # instance index for your data generation
success_cnt = 0
fail_cnt = 0
bounds = (0., None)
max_iter = 15000


data_t = 'test'        # 'train'/'valid'/test'


if data_t == 'train': 
    STD = train_std
elif data_t == 'valid':
    STD = valid_std
else:
    STD = test_std
# STD size: (# of instance, # of pairs, std values)   

MLU = []
graph_info = []
fail_idx = []
success_idx = []
var = []
As = []
bs = []
cs = []
for g in range(len(group)):
    stds = []
    ips = []
    success_cnt = 0
    times = []
    for n in range(STD.shape[0]): 
        std = list(STD[n])
        
        A1, b1, A2, b2, c = generate_reallocation(group[g], std, Pd, k)
        A = np.vstack([A1, A2])
        b = np.hstack([b1, b2])
        print(A.shape, b.shape, c.shape)
        As.append(A)
        bs.append(b)
        cs.append(c)
        n_time = time.time()
        try:
            A_eq = None
            b_eq = None
            A_ub = A
            b_ub = b
            res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, 
                          method='interior-point')
            times.append(time.time()-n_time)
            print(res)
            var.append(res.x)
            MLU.append(res.x[-1])
            success_idx.append(n)
            
        except (LinAlgWarning, OptimizeWarning, AssertionError):
            fail_cnt += 1
            fail_idx.append(n)
            continue
        else:
            if res.success and not np.isnan(res.fun):
                ips.append((torch.from_numpy(A).to(torch.float), torch.from_numpy(b).to(torch.float), torch.from_numpy(c).to(torch.float)))
                success_cnt += 1
                stds.append(std)


    with open(root+'/raw/instance_'+str(pkg_idx)+'_stds.pkl','wb') as f:
        pickle.dump(stds, f)
    with gzip.open(f'{root}/raw/instance_{pkg_idx}.pkl.gz', "wb") as file:
        pickle.dump(ips, file)
    pkg_idx += 1

    graph_info.append((group[g].number_of_nodes(), group[g].number_of_edges(), sum(times)/len(times)))

if data_t == 'train': 
    np.save(root+'/raw/abilene_train_info', graph_info)
    np.save(root+'/raw/abilene_train_mlu', MLU)
    for i in graph_info:
        print('Graph info and average time used:', i)
elif data_t == 'valid':
    np.save(root+'/raw/abilene_valid_info', graph_info)
    np.save(root+'/raw/abilene_valid_mlu', MLU)
    for i in graph_info:
        print('Graph info and average time used:', i)
else: 
    np.save(root+'/raw/abilene_test_info', graph_info)
    np.save(root+'/raw/abilene_test_mlu', MLU)
    for i in graph_info:
        print('Graph info and average time used:', i)
    
    
warnings.resetwarnings()

(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3500527474439895
            x: [ 1.405e-01  1.405e-01 ...  2.451e-02  3.501e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3499991032270663
            x: [ 1.403e-01  1.403e-01 ...  2.543e-02  3.500e-01]
          nit: 10
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.32915790647858784
            x: [ 1.405e-01  1.405e-01 ...  2.128e-02  3.292e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3380495722283207
            x: [ 1.404e-01  1.404e-01 ...  2.314e-02  3.380e-01]
          nit: 9
 intermediat

      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.34375548070521705
            x: [ 1.407e-01  1.407e-01 ...  2.733e-02  3.438e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.34580654409064726
            x: [ 1.407e-01  1.407e-01 ...  2.601e-02  3.458e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.34575182807813754
            x: [ 1.407e-01  1.407e-01 ...  2.445e-02  3.458e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3387818526903673
            x: [ 1.407e-01  1.407e-01 ...  2.665e-02  3.388e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (

(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.327255261297495
            x: [ 1.408e-01  1.408e-01 ...  2.110e-02  3.273e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3167475236591471
            x: [ 1.407e-01  1.407e-01 ...  2.468e-02  3.167e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.31198862906701785
            x: [ 1.407e-01  1.407e-01 ...  2.428e-02  3.120e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.326247404317622
            x: [ 1.406e-01  1.406e-01 ...  2.441e-02  3.262e-01]
          nit: 9
 intermediate: 

      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 2.372221453870807
            x: [ 1.557e-01  1.557e-01 ...  8.549e-02  2.372e+00]
          nit: 12
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 2.1799024709732864
            x: [ 1.538e-01  1.538e-01 ...  9.639e-02  2.180e+00]
          nit: 12
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 1.9633726428328027
            x: [ 1.511e-01  1.511e-01 ...  9.344e-02  1.963e+00]
          nit: 10
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 1.7653162959385202
            x: [ 1.503e-01  1.503e-01 ...  9.141e-02  1.765e+00]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1

(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3256427688357549
            x: [ 1.406e-01  1.406e-01 ...  4.773e-02  3.256e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3254528075377232
            x: [ 1.406e-01  1.406e-01 ...  3.713e-02  3.255e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.2527299000051314
            x: [ 1.402e-01  1.402e-01 ...  3.264e-02  2.527e-01]
          nit: 10
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.2557347426564389
            x: [ 1.403e-01  1.403e-01 ...  4.170e-02  2.557e-01]
          nit: 9
 intermediate

...

      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3429523140057897
            x: [ 1.404e-01  1.404e-01 ...  1.646e-02  3.430e-01]
          nit: 10
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3512273463230934
            x: [ 1.401e-01  1.401e-01 ...  4.793e-03  3.512e-01]
          nit: 11
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3689497829987279
            x: [ 1.406e-01  1.406e-01 ...  3.142e-02  3.689e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3706921142823892
            x: [ 1.406e-01  1.406e-01 ...  3.284e-02  3.707e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1

(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3586881021526332
            x: [ 1.406e-01  1.406e-01 ...  5.698e-02  3.587e-01]
          nit: 10
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.36861524408803176
            x: [ 1.406e-01  1.406e-01 ...  4.567e-02  3.686e-01]
          nit: 12
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.37276540864289776
            x: [ 1.406e-01  1.406e-01 ...  5.495e-02  3.728e-01]
          nit: 11
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3739513161418206
            x: [ 1.405e-01  1.405e-01 ...  4.539e-02  3.740e-01]
          nit: 11
 interme

(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.42480495699577864
            x: [ 1.402e-01  1.402e-01 ...  3.221e-02  4.248e-01]
          nit: 11
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.4322492946272186
            x: [ 1.405e-01  1.405e-01 ...  3.401e-02  4.322e-01]
          nit: 10
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.44860336473481627
            x: [ 1.405e-01  1.405e-01 ...  4.848e-02  4.486e-01]
          nit: 10
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.439428000023461
            x: [ 1.403e-01  1.403e-01 ...  4.570e-02  4.394e-01]
          nit: 11
 intermed

      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.4453029118027899
            x: [ 1.412e-01  1.412e-01 ...  3.301e-02  4.453e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.45278965859452563
            x: [ 1.411e-01  1.411e-01 ...  3.630e-02  4.528e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.45315113985653166
            x: [ 1.412e-01  1.412e-01 ...  3.375e-02  4.532e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.445560955082282
            x: [ 1.411e-01  1.411e-01 ...  3.773e-02  4.456e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (10

      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.4379325987724431
            x: [ 1.409e-01  1.409e-01 ...  3.343e-02  4.379e-01]
          nit: 9
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.41116973657473116
            x: [ 1.404e-01  1.404e-01 ...  3.413e-02  4.112e-01]
          nit: 10
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.4069569914133543
            x: [ 1.405e-01  1.405e-01 ...  3.162e-02  4.070e-01]
          nit: 10
 intermediate: []
(162, 1057) (162,) (1057,)
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.40862093450268805
            x: [ 1.404e-01  1.404e-01 ...  3.130e-02  4.086e-01]
          nit: 10
 intermediate: []
(162, 1057) (162,)

In [51]:
var[0].shape, var[1].shape

((1057,), (1057,))

In [52]:
for i in range(int(1056/8)):
    print(np.sum(var[1][8*i:8*(i+1)]))

1.126434002603333
1.112334622984426
1.1155616783974898
1.1076788502619102
1.1118380756001345
1.1189963089461608
1.108381112760969
1.1237206856426807
1.1264814235702094
1.1160462970675173
1.118056012249752
1.1264411385193474
1.058872994001425
1.0992201298967557
1.0927649259876153
1.0815592654652084
1.1137951674789295
1.084269821306437
1.0912136265350294
1.108955920280569
1.1008001149685278
1.0987493048998607
1.0554265474075708
1.0326401942229055
1.0233316552809508
1.0399358847932623
1.0081434956461592
1.041663007571609
1.0130342132681074
1.1025470247607436
1.0760522385466942
1.0522666721722094
1.1032852186723325
1.1133269315895409
1.104701602191702
1.0336023541210193
1.1164525132600986
1.0993755191952341
1.1064900225524226
1.1115190454686859
1.095365133969223
1.1126886630950543
1.114000021163892
1.0991481025293313
1.0975619610092169
1.098699071707165
1.0598443231651358
1.1147777909259269
1.1028024246940182
1.1144432549348073
1.104497945402203
1.0868155440842542
1.1073076770285333
1.1075

In [48]:
As[0].shape, bs[0].shape, cs[0].shape, As[1].shape, bs[1].shape, cs[1].shape

((162, 1057), (162,), (1057,), (162, 1057), (162,), (1057,))

In [49]:
As[0], As[1]

(array([[-1.        , -1.        , -1.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        , -1.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.02400016,
          0.02400016, -1.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        , -1.        ]]),
 array([[-1.        , -1.        , -1.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [43]:
As[1][-2]

array([-0.        , -0.        , -0.        , ..., -0.02473167,
       -0.02473167,  1.        ])

In [101]:
var[1]

array([0.6347054 , 0.6347054 , 0.6347054 , ..., 0.09446552, 0.06540491,
       1.02912085])

In [104]:
np.matmul(As[1][-2], var[1])-bs[1][-2]

2.392317381595694

In [105]:
for i in range(int(1056/8)):
    print(var[1][8*i:8*(i+1)], np.sum(var[1][8*i:8*(i+1)]))

[0.6347054 0.6347054 0.6347054 0.6347054 0.6347054 0.6347054 0.6347054
 0.6347054] 5.0776431755818585
[0.63336229 0.63439023 0.62969734 0.61915552 0.61909693 0.63336229
 0.63336229 0.63336229] 5.03578916081879
[0.62547211 0.62383523 0.62279954 0.62278002 0.42690237 0.62100459
 0.62098322 0.42269051] 4.586467595169557
[0.62837973 0.62707155 0.4903018  0.62274469 0.62269903 0.48299706
 0.48291822 0.62837973] 4.585491809985825
[0.63469488 0.63469488 0.63469488 0.63469488 0.63469488 0.63469488
 0.63469488 0.63469488] 5.077559077166557
[0.63469488 0.63469488 0.63469488 0.63469488 0.63469488 0.63469488
 0.63469488 0.63469488] 5.077559077201642
[0.58072352 0.56716723 0.57948246 0.56720161 0.5790149  0.56669975
 0.14616868 0.1461718 ] 3.7326299603064923
[0.63464733 0.63408402 0.63322455 0.63081383 0.63080601 0.63464733
 0.63464733 0.63464733] 5.067517703953034
[0.63469488 0.63469488 0.63469488 0.63469488 0.63469488 0.63469488
 0.63469488 0.63469488] 5.07755907717752
[0.62148591 0.61847678 0.61

In [95]:
con1 = np.matmul(As[1], var[1])-bs[1]
con1.shape, con1[:132], con1[-30:]

((162,),
 array([-4.07764318, -4.03578916, -3.5864676 , -3.58549181, -4.07755908,
        -4.07755908, -2.73262996, -4.0675177 , -4.07755908, -3.6380162 ,
        -3.75310605, -4.07763665, -2.17003409, -0.73915014, -0.49816151,
        -1.83373419, -3.15660307, -0.28186659, -2.99629994, -3.54591888,
        -1.44822282, -1.81461688, -2.05072161, -0.24335229, -0.20226054,
        -0.22685768, -0.20350715, -0.24772616, -0.18686073, -0.30664469,
        -0.33950963, -0.24127964, -1.00942453, -3.60552539, -2.32285586,
        -1.12695166, -2.01458381, -0.71881225, -1.37843382, -0.52793561,
        -0.68869339, -2.52216035, -2.69552166, -0.68057344, -1.80879459,
        -2.10600196, -2.07995038, -2.91360826, -2.89097844, -3.48685735,
        -0.71979384, -2.37204074, -3.80096592, -2.12395228, -1.73974181,
        -3.60451837, -1.00430054, -2.04926798, -0.45344225, -1.99328252,
        -0.61218679, -0.21759085, -0.73858548, -1.80660633, -0.31853981,
        -2.0490269 , -4.07755908, -3.19753

In [19]:
import pickle
import gzip
import torch
import numpy as np

with gzip.open('raw/raw/abilene_leq1/instance_2.pkl.gz', 'rb') as f:
    data_test = pickle.load(f)
with gzip.open('raw/raw/abilene_leq1/instance_1.pkl.gz', 'rb') as f:
    data_valid = pickle.load(f)
with gzip.open('raw/raw/abilene_leq1/instance_0.pkl.gz', 'rb') as f:
    data_train = pickle.load(f)

In [20]:
data_test_idx = np.load('raw/raw/abilene_leq1/abilene_test_mlu.npy')
data_valid_idx = np.load('raw/raw/abilene_leq1/abilene_valid_mlu.npy')
data_train_idx = np.load('raw/raw/abilene_leq1/abilene_train_mlu.npy')

In [21]:
len(data_train), data_train_idx.shape

(11041, (12095,))

In [22]:
len(data_valid), data_valid_idx.shape

(1968, (2016,))

In [23]:
len(data_test), data_test_idx.shape

(1840, (2016,))

In [220]:
# raw/raw/abilene_leq-1
# instance_0/1/2: train/valid/test

In [ ]:
#python3 frameworks/gurobi_mlu.py --num_paths_per_pair 8 --opt_start_idx 0 --opt_end_idx 1 --topo abilene --framework gurobi